In [2]:
!pip install netCDF4
!pip install xarray

     |████████████████████████████████| 4.3 MB 15.5 MB/s eta 0:00:01
     |████████████████████████████████| 287 kB 55.2 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 720 kB 13.6 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [3]:
import xarray as xr
import requests
import netCDF4
import boto3
import matplotlib.pyplot as plt
import datetime as dt
import fsspec
import s3fs

In [4]:
lyr = 2017 # year
idyjl = 210 # day of year
d = dt.datetime(lyr,1,1) + dt.timedelta(days=idyjl)

fs = s3fs.S3FileSystem(anon=True) # connect to s3 bucket!

# create strings for the year and julian day
imon,idym=d.month,d.day
syr,sjdy,smon,sdym = str(lyr).zfill(4),str(idyjl).zfill(3),str(imon).zfill(2),str(idym).zfill(2)
    
# use glob to list all the files in the directory, for goes-east
file_location,var = fs.glob('s3://noaa-goes16/ABI-L2-CMIPC/'+syr+'/*/*/*.nc'),'SST'

In [5]:
%timeit
file_ob = [fs.open(file) for file in file_location] # open connection to files
file_ob

[<File-like object S3FileSystem, noaa-goes16/ABI-L2-CMIPC/2017/059/00/OR_ABI-L2-CMIPC-M3C01_G16_s20170590002504_e20170590005277_c20170590005335.nc>,
 <File-like object S3FileSystem, noaa-goes16/ABI-L2-CMIPC/2017/059/00/OR_ABI-L2-CMIPC-M3C01_G16_s20170590007504_e20170590010277_c20170590010337.nc>,
 <File-like object S3FileSystem, noaa-goes16/ABI-L2-CMIPC/2017/059/00/OR_ABI-L2-CMIPC-M3C01_G16_s20170590012504_e20170590015277_c20170590015338.nc>,
 <File-like object S3FileSystem, noaa-goes16/ABI-L2-CMIPC/2017/059/00/OR_ABI-L2-CMIPC-M3C01_G16_s20170590017505_e20170590020277_c20170590020335.nc>,
 <File-like object S3FileSystem, noaa-goes16/ABI-L2-CMIPC/2017/059/00/OR_ABI-L2-CMIPC-M3C01_G16_s20170590022505_e20170590025277_c20170590025337.nc>,
 <File-like object S3FileSystem, noaa-goes16/ABI-L2-CMIPC/2017/059/00/OR_ABI-L2-CMIPC-M3C01_G16_s20170590027505_e20170590030277_c20170590030336.nc>,
 <File-like object S3FileSystem, noaa-goes16/ABI-L2-CMIPC/2017/059/00/OR_ABI-L2-CMIPC-M3C01_G16_s201705900

In [8]:
import pandas as pd

In [ ]:
df_2017 = pd.DataFrame(file_ob)

In [7]:
file_ob[2].read()

b''

In [8]:
ds = xr.open_dataset(file_ob[0].read())

ValueError: can't open netCDF4/HDF5 as bytes try passing a path or file-like object

In [ ]:
ds = netCDF4.Dataset(fs.cat_file(file_ob[0]), 'r')

In [ ]:
ds = xr.open_mfdataset(file_ob,combine='nested',concat_dim='time') #note file is super messed up formatting